In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install quandl

In [ ]:
#imports
import pandas as pd
import os
import numpy as np
import quandl
import regex as re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#quandl.ApiConfig.api_key="Insert your key"


# Obtaining data

In [ ]:
cd /kaggle/input/sg-sectoral-nasdaq-indices

In [ ]:
#Get list of available indices
f=open("SG indices.txt","r")
indices=f.read()
f.close()
print(indices)

## Make API calls and combine data
### changing to markdown because you need to have a Quandl API key to run
pat="Singapore ([\w\s]+)\W(NQSG\d+\w*)\W(?:\n|(NQSG\d+\w*))"
indexlist=re.findall(pat,indices)
for item in indexlist:
    if item[2]=="":
        print(item[1])
        df1=quandl.get(f"NASDAQOMX/{item[1]}")
        df1.to_csv(f"{item[0]}.csv",index=False)
    else:
        print(item[1])
        df1=quandl.get(f"NASDAQOMX/{item[1]}")
        df2=quandl.get(f"NASDAQOMX/{item[2]}")
        newdf=pd.concat([df1,df2])
        newdf.to_csv(f"{item[0]}.csv",index=False)

In [ ]:
cd /kaggle/input/sg-sectoral-nasdaq-indices/2013 minimum/2013 min

## Create Combined dataframe

In [ ]:
#get a master dataframe
masterdf=pd.read_csv("Financials Index.csv",usecols=["Trade Date"])
masterdf.sort_values("Trade Date",inplace=True,ignore_index=True)
masterdf.head()

for a,b,c in os.walk("/kaggle/input/sg-sectoral-nasdaq-indices/2013 minimum/2013 min"):
    itemlist=c
#not sure why the above is not working in kaggle (works in jupyter) but anyways:
itemlist=['Basic Materials Index.csv', 'Consumer Services Index.csv', 'Consumer Services Large Mid Cap Index.csv', 'Energy Index.csv', 'Financials Index.csv', 'Financials Large Mid Cap Index.csv', 'Health Care Index.csv', 'Industrials Index.csv', 'Industrials Large Mid Cap Index.csv', 'Original.txt', 'SG indices.txt', 'Technology Index.csv', 'Utilities Index.csv']

for item in itemlist:
    if item[-4:] == ".csv":
        df=pd.read_csv(item,usecols=["Trade Date","Index Value"])
        df.sort_values("Trade Date",inplace=True,ignore_index=True)
        df.rename(columns={"Index Value":item[:-4]},inplace=True)
        masterdf=pd.merge(masterdf,df,on=["Trade Date"])

#to check whether all are included
masterdf.head()
#to save
# #masterdf.to_csv("Combined.csv",index=False)

## To clean up saved CSVs, optional

itemlist=["Combined.csv",'Basic Materials Index.csv', 'Consumer Services Index.csv', 'Consumer Services Large Mid Cap Index.csv', 'Energy Index.csv', 'Financials Index.csv', 'Financials Large Mid Cap Index.csv', 'Health Care Index.csv', 'Industrials Index.csv', 'Industrials Large Mid Cap Index.csv', 'Technology Index.csv', 'Utilities Index.csv']

for item in itemlist:
    df=pd.read_csv(item)
    #in case of any unnecessary index
    df.drop(["Unnamed: 0","Unnamed: 0.1","Unnamed: 0.1.1","Date"],axis=1,inplace=True,errors="ignore")
    df.sort_values("Trade Date",inplace=True,ignore_index=True)
    df["Date"]=pd.to_datetime(df["Trade Date"],infer_datetime_format=True)
    df.to_csv(item)

# Plotting

## Correlation matrix plotting

In [ ]:
#to plot a triangular correlation matrix (only for those 2013 minimum)
masterdf=pd.read_csv('Combined.csv')
#create baseplot
fig, ax=plt.subplots(figsize=(12,12))
plt.title(label="SG NASDAQ Indices Correlation Matrix")


#too cluttered to type out in the params
collist=['Trade Date', 'Basic Materials Index', 'Consumer Services Index',
       'Consumer Services Large Mid Cap Index', 'Energy Index',
       'Financials Index', 'Financials Large Mid Cap Index',
       'Health Care Index', 'Industrials Index',
       'Industrials Large Mid Cap Index', 'Technology Index',
       'Utilities Index']

#for triangle corr matrix
mask=np.triu(masterdf[collist].corr())
sns.heatmap(masterdf[collist].corr(),annot=True,annot_kws={"size": 7},square=True,cmap="coolwarm",mask=mask)

#annotation
ax.add_patch(plt.Rectangle((4,6),2,1,fill=False,lw=2))
#ax.add_patch(plt.Rectangle((4,8),2,1,fill=False,lw=2))
#ax.add_patch(plt.Rectangle((2,9),2,1,fill=False,lw=2))
#ax.add_patch(plt.Rectangle((4,10),3,1,fill=False,lw=2))
#plt.axhline(y=7,xmin=0,xmax=0.64,ls="--",c="purple")
#plt.axvline(x=7,ymin=0,ymax=0.36,ls="--",c="purple")

#final formatting
plt.xticks(rotation=45,size=7,horizontalalignment="right")
plt.yticks(size=7)
#plt.tight_layout(h_pad=-1.5)

## Single and multiple indices plotting

In [ ]:
#To plot single or multiple indices
def indexplot(index="Financials",lm=False,hl=False,sma=30,start="",end=""):
    if lm==True:
        name=index+" Large Mid Cap Index"
    if lm==False:
        name=index+" Index"
    
    df=pd.read_csv(f"{name}.csv")
    df.rename(columns={"Trade Date":"XLDate","Index Value":"Value","Total Market Value":"MarketValue","Dividend Market Value":"Dividend"},inplace=True)
    df.sort_values("XLDate",inplace=True,ignore_index=True)
    df["Date"]=pd.to_datetime(df["XLDate"],infer_datetime_format=True)
    
    #future: add the start and end date functionality using df.loc
    
    #base plotting code
    fig,ax=plt.subplots(figsize=(12,8))    
    plt.title(name)
    plt.grid(axis="y",alpha=0.5,linestyle="--")
    plt.ylabel("Index Value")
    plt.xlabel("Date")
    plt.xticks(rotation=30)
    
    #plotting
    sns.lineplot(x="Date",y="Value",data=df,alpha=1,label=f"{name}")
    #for high and low
    if hl==True:
        sns.lineplot(x="Date",y="High",data=df,alpha=0.5,label=f"{name} High")
        sns.lineplot(x="Date",y="Low",data=df,alpha=0.5,label=f"{name} Low")
        
    #SMA30 plot
    if sma != False:
        df["SMA"]=df.Value.rolling(window=sma).mean()
        sns.lineplot(x="Date",y="SMA",data=df,alpha=0.8,linestyle="--",label=f"{name} SMA{sma}")
    
    #annotations
    #auto add markers where there are dividends
    #ax.add_patch(plt.Rectangle((1,7),3,1,fill=False,lw=2)) 
    #plt.Circle((Test date,Test value),radius=1)
    #plt.errorbar to add the bollinger bands based on std dev?
    
    #legend
    plt.legend()
    plt.tight_layout()

    
    
def multiplot(names=["Financials"],lm=False,sma=30,start="",end="",label="SG NASDAQ Indices"):
    #removing hl=True/False because too cluttered as additional lines

    #future: add the start and end date functionality using df.loc
    
    #base plotting code
    fig,ax=plt.subplots(figsize=(12,8))    
    plt.title(label)
    plt.grid(axis="y",alpha=0.5,linestyle="--")
    plt.ylabel("Index Value")
    plt.xlabel("Date")
    plt.xticks(rotation=30)
    
    for item in names:        
        if lm==True:
            name=item+" Large Mid Cap Index"
        if lm==False:
            name=item+" Index"
        #gather data
        df=pd.read_csv(f"{name}.csv")
        df.rename(columns={"Trade Date":"XLDate","Index Value":"Value","Total Market Value":"MarketValue","Dividend Market Value":"Dividend"},inplace=True)
        df.sort_values("XLDate",inplace=True,ignore_index=True)
        df["Date"]=pd.to_datetime(df["XLDate"],infer_datetime_format=True)
        
        #plotting
        sns.lineplot(x="Date",y="Value",data=df,alpha=1,label=f"{name}")
        
        #hashing because too cluttered
        #if hl==True:
        #    sns.lineplot(x="Date",y="High",data=df,alpha=0.5,label=f"{name} High")
        #    sns.lineplot(x="Date",y="Low",data=df,alpha=0.5,label=f"{name} Low")
        
        #SMA30 plot
        if sma != False:
            if sma==True:
                sma=5
            df["SMA"]=df.Value.rolling(window=sma).mean()
            sns.lineplot(x="Date",y="SMA",data=df,alpha=0.8,linestyle="--",label=f"{name} SMA{sma}")
    
    #annotations
    #auto add markers where there are dividends
    #ax.add_patch(plt.Rectangle((1,7),3,1,fill=False,lw=2)) 
    #plt.Circle((Test date,Test value),radius=1)
    #plt.errorbar to add the bollinger bands based on std dev?
    
    #legend
    plt.legend()
    plt.tight_layout()

### Sample graphs

In [ ]:
indexplot("Utilities",lm=False,hl=False,sma=30)

In [ ]:
multiplot(["Financials","Industrials","Consumer Services","Utilities","Health Care","Technology"],lm=False,sma=False)

##### Hope this helped!